In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym

import numpy as np
import pandas as pd

import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from functools import reduce
import collections
step = collections.namedtuple('step', ('obs', 'act_v', 'act', 'next_obs', 'rew', 'done'))

In [3]:
# GAME = 'CartPole-v1'
GAME = 'Pendulum-v0'

In [4]:
env = gym.make(GAME)
env._max_episode_steps = 500
obs_n = env.observation_space.shape[0]
act_n = env.action_space.shape[0]

In [5]:
class Memory():
    def __init__(self, max_size):
        self.data = collections.deque(maxlen=max_size)
        self.prio = collections.deque(maxlen=max_size)
        
    def append(self, data):
        self.data.append(data)
        if not len(self.prio):
            self.prio.append(1)
        else:
            self.prio.append(np.array(self.prio).max())
        
    def extend(self, data):
        self.data.extend(data)
        if not len(self.prio):
            self.prio.extend([1]*len(data))
        else:
            self.prio.extend([np.array(self.prio).max()]*len(data))
        
    def __len__(self):
        return len(self.data)
    
    def apply_prio(self, prio, indices):
        for i, p in zip(indices, prio):
            self.prio[i] = p
        
        enough = np.where(np.array(self.prio) >= 1e-8)
        
        data_left = np.array(self.data)[enough]
        prio_left = np.array(self.prio)[enough]
        
        self.data.clear()
        self.prio.clear()
        
        self.data.extend(data_left)
        self.prio.extend(prio_left)
        
    def sample(self, size, prio_decay):
        assert size<=len(self)
        assert len(self) == len(self.prio)
        prio = np.power(self.prio, min(prio_decay,1))
        prob = prio/prio.sum()
        
        indices = np.random.choice(len(self), size, replace=False, p=prob)
        sample = np.array(self.data)[indices]
        weight = np.array(prob)[indices]**-0.1
        weight = weight/weight.sum()

        return sample, indices, weight

In [6]:
def get_steps(env, net, gamma):
    obs = env.reset()
    scenario = []
    rew_sum = 0
    
    while True:
        with torch.no_grad():
            act_v = net(torch.FloatTensor([obs]).to(device)).cpu()*2
            noise = torch.FloatTensor(np.random.normal(0,1, size=act_n) * np.exp(-count/40))
            act_v += noise
            act = act_v.numpy()[0].clip(-2,2)
#             act = act_v.argmax().item()
            
        next_obs, rew, done, _ = env.step(act)
        rew += 10
        rew /= 10
        rew_sum += rew
    
        data = step(obs, act_v.numpy()[0], act, next_obs, rew, done)
        scenario.append(data)
        
        if done:
            print(len(scenario),rew_sum)
            next_obs=env.reset()
            scenario.clear()
            rew_sum = 0
            
        obs = next_obs
        yield data

In [7]:
class Actor(nn.Module):
    def __init__(self, hidden = 128):
        super(Actor, self).__init__()
        self.actor = nn.Sequential(
            nn.Linear(obs_n, hidden),
            nn.ReLU(),
            nn.Linear(hidden, int(hidden/2)),
            nn.ReLU(),
            nn.Linear(int(hidden/2), act_n),
            nn.Tanh()
        )
        
    def alph_update(self, net, alpha):
        for bhv, tgt in zip(net.parameters(), self.parameters()):
            tgt.data.copy_(bhv.data*alpha + tgt.data*(1-alpha))
        self.eval()
        
    def forward(self, x):
        return self.actor(x)
    
class Critic(nn.Module):
    def __init__(self, hidden = 128):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_n, hidden),
            nn.ReLU(),
        )
        self.critic = nn.Sequential(
            nn.Linear(hidden + act_n, int(hidden/2)),
            nn.ReLU(),
            nn.Linear(int(hidden/2), 1)
        )
        
    def alph_update(self, net, alpha):                
        for bhv, tgt in zip(net.parameters(), self.parameters()):
            tgt.data.copy_(bhv.data*alpha + tgt.data*(1-alpha))
        self.eval()
        
    def forward(self, x, act_v):
        x = torch.cat([self.net(x), act_v], dim=1)
        return self.critic(x)

In [8]:
actor = Actor().to(device)
actor_target = Actor().to(device)
actor_target.load_state_dict(actor.state_dict())
actor_target.eval()

critic = Critic().to(device)
critic_target = Critic().to(device)
critic_target.load_state_dict(critic.state_dict())
critic_target.eval()

ACT_LR = 0.0001
CRT_LR = 0.0004
ALPHA = 0.05
GAMMA = 0.998
epoch = 0
EPOCH = 1000
PRINT_STEP = 2000
actor_optim = optim.Adam(actor.parameters(), lr = ACT_LR)
critic_optim = optim.Adam(critic.parameters(), lr = CRT_LR)

agent = get_steps(env, actor, GAMMA)

MEM_SIZE = 20000
MEM_INIT = 2000
MEM_BATCH = 64

storage = Memory(MEM_SIZE)
while len(storage) < MEM_INIT:
    storage.append(next(agent))

500 133.6746909624773
500 95.99051064672173
500 124.11777254275269
500 132.62630885983558


In [ ]:
count = 0
for epoch in range(EPOCH):
    print("epoch #%d] "%epoch)
    a_loss, c_loss = 0, 0
    for _ in range(PRINT_STEP):
        count += 1
        storage.append(next(agent))
        sample, indices, weight = storage.sample(MEM_BATCH, np.exp(-count/MEM_SIZE))
        weight = torch.FloatTensor(weight).to(device)
        
        batch = step(*list(zip(*sample)))
        
        obs = torch.FloatTensor(batch.obs).to(device)
        act_v = torch.FloatTensor(batch.act_v).to(device)
        act = torch.LongTensor(batch.act).to(device).unsqueeze(1)
        next_obs = torch.FloatTensor(batch.next_obs).to(device)
        rew = torch.FloatTensor(batch.rew).to(device).unsqueeze(1)
        done = torch.BoolTensor(batch.done).to(device)

        critic_optim.zero_grad()
        q_pred = critic(obs, act_v)

        q_next = critic_target(next_obs, actor_target(next_obs))
        q_next[done] = 0.
        q_target = rew + GAMMA * q_next.detach()
        
        new_prio = ((q_pred - q_target) ** 2).sum(1) ** 1/2
        critic_loss = (new_prio * weight).mean()
        critic_loss.backward()
        critic_optim.step()

        actor_optim.zero_grad()
        q = critic(obs, actor(obs))
        actor_loss= -q.mean()
        actor_loss.backward()
        actor_optim.step()
        
        a_loss += actor_loss.cpu().item()
        c_loss += critic_loss.cpu().item()

        critic_target.alph_update(critic, ALPHA)
        actor_target.alph_update(actor, ALPHA)
        
        storage.apply_prio(new_prio.cpu().data.numpy(), indices)
    print()
    print("%.7f %.7f"%(a_loss/PRINT_STEP, c_loss/PRINT_STEP))

epoch #0] 
131 66.57816408818363
500 265.731213285654
500 260.9183294704573
500 262.3752512213582

-17.3349284 0.2931088
epoch #1] 
500 243.7491536837168
500 298.0846538926196
500 281.3922648458556
500 249.61500585551696

-19.2311809 0.2824427
epoch #2] 
500 285.80921863329746
500 279.1745613822389
500 256.2852258072362
500 272.7941167676045

-21.6212914 0.3047933
epoch #3] 
500 297.4800479734764
500 271.0320437020152
500 323.0520676141691
500 325.40445844403996

-25.4099127 0.3811357
epoch #4] 
500 314.7171891109016
500 300.87644048956787
500 326.66135896354166
500 336.81179678974166

-27.5798442 0.3992145
epoch #5] 
500 336.9628004651763
500 336.45996791066074
500 327.6095824596635
500 325.0786910227067

-28.2355120 0.3369050
epoch #6] 
500 336.34231167889305
500 336.0295115083042
500 337.3977613657129
500 331.8091051113123

-31.7849457 0.3585982
epoch #7] 
500 335.975620546981
500 323.4551623123097
500 323.4952398143114
500 347.9970355258368

-34.8964980 0.3726862
epoch #8] 
500 337

In [ ]:
q_target

In [ ]:
a = np.power(storage.prio, max(np.exp(-epoch/100),1))

In [ ]:
a.min()

In [14]:
actor.eval()

obs = env.reset()
env.render()

while True:
    
    with torch.no_grad():
        act_v = actor(torch.FloatTensor([obs]).to(device)).cpu()
#         act = act_v.argmax().item()
        act = act_v.numpy()[0].clip(-1,1)
    next_obs, _, done, _ = env.step(act)
    obs = next_obs
    env.render()
    
    print(act_v, q)
    if done:
        break

env.close()
actor.train()
print()

tensor([[0.9986]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [20.1989],
        [18.1337],
        [21.6355],
        [16.

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [2

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [20

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [2

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [2

...

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.0000]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
      

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [2

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.9972]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
       

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.0000]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
      

        [15.4834]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-1.]]) tensor([[20.5009],
        [27.6052],
        [11.5486],
        [16.6054],
        [48.4818],
        [18.0750],
        [27.0519],
        [20.8619],
        [18.6292],
        [22.1998],
        [34.3325],
        [27.0388],
        [21.4230],
        [26.8252],
        [18.3234],
        [23.3736],
        [28.3120],
        [19.4951],
        [26.3377],
        [16.2474],
        [15.5046],
        [16.5658],
        [16.1338],
        [30.0587],
        [17.6996],
        [26.0921],
        [21.3785],
        [32.2970],
        [34.4961],
        [13.0989],
        [24.3360],
        [11.5618],
        [29.1478],
        [22.4340],
        [-7.0039],
        [26.7524],
        [ 6.4919],
        [13.1705],
        [18.9718],
        [17.6163],
        [19.5799],
        [21.0276],
        [19.8143],
        [22.8984],
        [29.1912],
        [29.3100],
        [29.7170],
        [27.5892],
        [2